### Specifying Glossary

In [ ]:
from google.cloud import translate_v3 as translate


def create_glossary(
    project_id: str = "numeric-chassis-395210",
    input_uri: str = "gs://mal_trans_glossary/glossary.csv",
    glossary_id: str = "mal-eng-glossary",
    timeout: int = 180,
) -> translate.Glossary:
    """
    Create a equivalent term sets glossary. Glossary can be words or
    short phrases (usually fewer than five words).
    https://cloud.google.com/translate/docs/advanced/glossary#format-glossary
    """
    client = translate.TranslationServiceClient()

    # Supported language codes: https://cloud.google.com/translate/docs/languages
    source_lang_code = "ml"
    target_lang_code = "en"
    location = "us-central1"  # The location of the glossary

    name = client.glossary_path(project_id, location, glossary_id)
    language_codes_set = translate.types.Glossary.LanguageCodesSet(language_codes=[source_lang_code, target_lang_code])
    # language_codes_pair = translate.Glossary.LanguageCodePair()
    # language_codes_pair.source_language_code = 'ml'
    # language_codes_pair.target_language_code = 'en'

    gcs_source = translate.types.GcsSource(input_uri=input_uri)

    input_config = translate.types.GlossaryInputConfig(gcs_source=gcs_source)

    glossary = translate.types.Glossary(
        name=name, language_codes_set=language_codes_set, input_config=input_config
    )

    parent = f"projects/{project_id}/locations/{location}"
    # glossary is a custom dictionary Translation API uses
    # to translate the domain-specific terminology.
    operation = client.create_glossary(parent=parent, glossary=glossary)

    result = operation.result(timeout)
    print(f"Created: {result.name}")
    print(f"Input Uri: {result.input_config.gcs_source.input_uri}")

    return result


In [ ]:
result = create_glossary()

Created: projects/129500516432/locations/us-central1/glossaries/mal-eng-glossary
Input Uri: gs://mal_trans_glossary/glossary.csv


## Deleting the glossary

In [ ]:
from google.cloud import translate_v3 as translate


def delete_glossary(
    project_id: str = "numeric-chassis-395210",
    glossary_id: str = "YOUR_GLOSSARY_ID",
    timeout: int = 180,
) -> translate.Glossary:
    """Delete a specific glossary based on the glossary ID.

    Args:
        project_id: The ID of the GCP project that owns the glossary.
        glossary_id: The ID of the glossary to delete.
        timeout: The timeout for this request.

    Returns:
        The glossary that was deleted.
    """
    client = translate.TranslationServiceClient()

    name = client.glossary_path(project_id, "us-central1", glossary_id)

    operation = client.delete_glossary(name=name)
    result = operation.result(timeout)
    print(f"Deleted: {result.name}")

    return result

In [ ]:
result = delete_glossary(glossary_id="mal-eng-glossary")
print(result)

Deleted: projects/129500516432/locations/us-central1/glossaries/mal-eng-glossary
name: "projects/129500516432/locations/us-central1/glossaries/mal-eng-glossary"
submit_time {
  seconds: 1696587629
  nanos: 906891030
}
end_time {
  seconds: 1696587630
  nanos: 28205086
}



## Translate

In [ ]:
# from google.cloud import translate_v3 as translate


def translate_text_with_glossary(
    text: str = "YOUR_TEXT_TO_TRANSLATE",
    project_id: str = "numeric-chassis-395210",
    glossary_id: str = "mal-eng-glossary",
) -> translate.TranslateTextResponse:
    """Translates a given text using a glossary.

    Args:
        text: The text to translate.
        project_id: The ID of the GCP project that owns the glossary.
        glossary_id: The ID of the glossary to use.

    Returns:
        The translated text."""
    client = translate.TranslationServiceClient()
    location = "us-central1"
    parent = f"projects/{project_id}/locations/{location}"

    glossary = client.glossary_path(
        project_id, "us-central1", glossary_id  # The location of the glossary
    )

    glossary_config = translate.TranslateTextGlossaryConfig(glossary=glossary)

    # Supported language codes: https://cloud.google.com/translate/docs/languages
    response = client.translate_text(
        request={
            "contents": [text],
            "target_language_code": "en",
            "source_language_code": "ml",
            "parent": parent,
            "glossary_config": glossary_config,
        }
    )

    print("Translated text: \n")
    for translation in response.glossary_translations:
        print(f"\t {translation.translated_text}")

    return translation.translated_text # return response.glossary_translations

In [ ]:
res = translate_text_with_glossary(text='''പട്ടികയിലെ വഹകൾ ചാർജ്ജായി യാതൊരു വിധ കടബാധ്യതകളും കോർട്ട് ജപ്തി മുതലായ ബാദ്ധ്യതകളും നികുതി ബാക്കിയും ഇല്ല.''')
# print(res)

Translated text: 

	 There are no debts, court foreclosures etc. and tax arrears against the listed properties.
